`IEnumerable<TSource>`→ `TSource`  

| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`First`, `FirstOrDefault` |Returns the first element in the sequence, optionally <br> satisfying a predicate |SELECT TOP 1 ... <br> ORDER BY ...
|`Last`,<br> `LastOrDefault` |Returns the last element in the sequence, optionally <br> satisfying a predicate| SELECT TOP 1 ... <br> ORDER BY ... DESC
|`Single`,<br> `SingleOrDefault` | Equivalent to First/FirstOrDefault, but <br> throws an exception if there is more than one match | 
| `ElementAt`, <br> `ElementAtOrDefault` | Returns the element at the specified position | Exception thrown
|`MinBy`, `MaxBy`| Returns the element with the smallest or largest <br> value | Exception thrown
| `DefaultIfEmpty` | Returns a single-element sequence whose value <br> is default(TSource) if the sequence has no<br> elements | OUTER JOIN

***Methods*** ending in `OrDefault` return ***default***(`TSource`) `rather than throwing an exception` if the input sequence is empty or if no elements match the supplied predicate.

***default(TSource)*** is `null` for `reference type` elements, `false` for the ***bool*** type, and `zero` for ***numeric types***.

### First, Last, and Single

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Predicate*** (optional) |`TSource => bool`

In [ ]:
int[] numbers = { 1, 2, 3, 4, 5 };
int first = numbers.First(); // 1
int last = numbers.Last(); // 5
int firstEven = numbers.First (n => n % 2 == 0); // 2
int lastEven = numbers.Last (n => n % 2 == 0); // 4

In [ ]:
int[] numbers = { 1, 2, 3, 4, 5 };

//The following demonstrates First versus FirstOrDefault:
int firstBigError = numbers.First (n => n > 10); // Exception
int firstBigNumber = numbers.FirstOrDefault (n => n > 10); // 0

In [ ]:
//Single requires exactly one matching element; 
//SingleOrDefault requires one or zero matching elements

int[] numbers = { 1, 2, 3, 4, 5 };

int onlyDivBy3 = numbers.Single (n => n % 3 == 0); // 3
int divBy2Err = numbers.Single (n => n % 2 == 0); // Error: 2 & 4 match

int singleError = numbers.Single (n => n > 10); // Error
int noMatches = numbers.SingleOrDefault (n => n > 10); // 0
int divBy2Error = numbers.SingleOrDefault (n => n % 2 == 0); // Error

In [ ]:
//In EF Core, Single is often used to retrieve a row from a table by primary key:

Customer cust = dataContext.Customers.Single (c => c.ID == 3);

### ElementAt

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Index of element to return***  |`int`

In [ ]:
int[] numbers = { 1, 2, 3, 4, 5 };
int third = numbers.ElementAt (2); // 3
int tenthError = numbers.ElementAt (9); // Exception
int tenth = numbers.ElementAtOrDefault (9); // 0

### MinBy and MaxBy

In [ ]:
//MinBy and MaxBy (introduced in .NET 6)

string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };
Console.WriteLine (names.MaxBy (n => n.Length)); // Harry

//If two or more elements share a minimum/maximum value, MinBy/MaxBy returns
//the first:
Console.WriteLine (names.MinBy (n => n.Length)); // Tom

### DefaultIfEmpty

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<ul><li><p><strong>اگر دنباله ورودی (input sequence) دارای عناصر باشد:</strong></p><ul><li>متد <code>DefaultIfEmpty</code> تغییری در آن ایجاد نمی‌کند و دنباله اصلی را بازمی‌گرداند.</li></ul></li><li><p><strong>اگر دنباله ورودی خالی باشد:</strong></p><ul><li>متد <code>DefaultIfEmpty</code> یک دنباله جدید با یک عنصر ایجاد می‌کند که مقدار آن برابر مقدار پیش‌فرض نوع (default value of type) است. مقدار پیش‌فرض هر نوع در C# به شکل زیر تعریف می‌شود:<ul><li>برای انواع مرجع (مثل <code>string</code>): <code>null</code></li><li>برای انواع مقدار (مثل <code>int</code> یا <code>double</code>): صفر (<code>0</code>)</li><li>برای نوع <code>bool</code>: مقدار <code>false</code></li></ul></li></ul></li></ul>
</div>

In [ ]:
//دنباله‌ای که دارای عناصر است
var numbers = new List<int> { 1, 2, 3 };
var result = numbers.DefaultIfEmpty(); // خروجی: [1, 2, 3]

In [ ]:
//دنباله‌ای که خالی است
var numbers = new List<int>();
var result = numbers.DefaultIfEmpty(); // خروجی: [0]

In [ ]:
// استفاده برای نوع مرجع
var strings = new List<string>();
var result = strings.DefaultIfEmpty(); // خروجی: [null]

In [ ]:
//استفاده از مقدار دلخواه
var numbers = new List<int>();
var result = numbers.DefaultIfEmpty(42); // خروجی: [42]

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>1. <strong>جلوگیری از بروز خطا در زمان اجرا</strong></h3>
<p>هنگامی که کوئری LINQ ممکن است یک دنباله خالی برگرداند، استفاده از <code>DefaultIfEmpty</code> باعث می‌شود به جای یک دنباله خالی، یک مقدار پیش‌فرض (یا مقدار مشخص شده) بازگردد. این کار از بروز خطا در پردازش‌های بعدی جلوگیری می‌کند.</p>
</div>

In [ ]:
var products = new List<string>(); // لیست خالی
var result = products.DefaultIfEmpty("No Products").ToList();

foreach (var item in result)
{
    Console.WriteLine(item); // خروجی: No Products
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>2. <strong>نمایش نتایج خالی در کوئری‌های پایگاه داده</strong></h3>
<p>زمانی که از LINQ to SQL یا LINQ to Entities استفاده می‌کنید، ممکن است بخواهید در صورتی که یک جدول یا نتیجه‌ای خالی باشد، یک مقدار پیش‌فرض را نمایش دهید.</p>
<h4>مثال:</h4>
<p>فرض کنید یک جدول <code>Orders</code> دارید و می‌خواهید در صورتی که هیچ سفارشی برای یک مشتری خاص وجود ندارد، یک مقدار پیش‌فرض را در نتیجه قرار دهید:</p>
</div>

In [ ]:
var customerOrders = db.Orders
    .Where(o => o.CustomerId == 1)
    .Select(o => o.OrderId)
    .DefaultIfEmpty(-1)
    .ToList();

foreach (var orderId in customerOrders)
{
    Console.WriteLine(orderId); // خروجی: -1 اگر سفارشی وجود نداشته باشد
}


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>3. <strong>ایجاد ترکیب Outer Join</strong></h3>
<p>در ترکیب‌های خارجی (Outer Join)، ممکن است نتیجه کوئری شامل دنباله‌های خالی باشد. با استفاده از <code>DefaultIfEmpty</code> می‌توانید برای دنباله‌های خالی، مقادیر پیش‌فرض قرار دهید.</p>
<h4>مثال:</h4>
<p>فرض کنید دو جدول <code>Customers</code> و <code>Orders</code> دارید. می‌خواهید لیستی از مشتریان به همراه سفارش‌های آن‌ها را نمایش دهید و اگر سفارشی وجود نداشت، مقدار پیش‌فرض نشان داده شود:</p>

</div>

In [ ]:
var query = from customer in db.Customers
            join order in db.Orders
                on customer.CustomerId equals order.CustomerId into ordersGroup
            from order in ordersGroup.DefaultIfEmpty()
            select new
            {
                CustomerName = customer.Name,
                OrderId = order?.OrderId ?? 0 // اگر سفارشی وجود نداشت، مقدار پیش‌فرض 0 باشد
            };

foreach (var item in query)
{
    Console.WriteLine($"Customer: {item.CustomerName}, OrderId: {item.OrderId}");
}
